In [ ]:
import csv
import random
import os, sys
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
from pandas.plotting import scatter_matrix

In [ ]:
#mode = "all_pos"
#attack = "FLO"

#mode = "4_pos"
mode = "JAM_always_attack"
attack = "JAM"

### Load and process BRIAN data

In [ ]:
# Stats files
if mode == "all_pos":
    stats_dir_FLO = "/data/Brian/csv_stats/FLO_all_positions/"
elif mode == "4_pos":
    stats_dir_FLO = "/data/Brian/csv_stats/FLO_4_positions/"
elif mode == "JAM_always_attack":
    stats_dir_JAM = "/data/Brian/csv_stats/JAM_27_positions_1-20n_10exec/"

# Raster aggregated ("cloud of dots") for each position
if attack == "FLO":
    stats_dir = stats_dir_FLO
    csv_rasters_aggr = stats_dir+"aggr_FLO.csv"
elif attack == "JAM":
    stats_dir = stats_dir_JAM
    csv_rasters_aggr = stats_dir+"27_positions.csv"

In [ ]:
stats_dir

In [ ]:
'''
df_aggr = pd.read_csv(csv_rasters_aggr, delimiter=";")
df_aggr["attack_n_neurons"] = df_aggr["attack"].astype("str") + " " + df_aggr["n_neurons"].astype("str")
list_n_neurons = df_aggr["n_neurons"].unique().tolist()
list_positions = df_aggr["position"].unique().tolist()
'''

#### Total number of spikes per position

In [ ]:
#df_total_spikes = df_aggr.copy()
'''
if mode == "all_pos":
    df_total_spikes = df_aggr[df_aggr.n_neurons.isin([0,5,35,55,75,105])].copy()
elif mode == "4_pos":
    df_total_spikes = df_aggr.copy()
else:
    df_total_spikes = df_aggr.copy()
'''

In [ ]:
'''
if mode == "all_pos":
    df_total_spikes = pd.DataFrame(df_total_spikes.groupby(["attack_n_neurons", "position_attack", "position", "n_exec"])["number_spikes"].sum())
elif mode == "4_pos":
    df_total_spikes = pd.DataFrame(df_total_spikes.groupby(["attack_n_neurons", "position_attack", "position", "v_increment", "n_exec"])["number_spikes"].sum())
else:
    df_total_spikes = pd.DataFrame(df_total_spikes.groupby(["attack_n_neurons", "position_attack", "position", "n_exec"])["number_spikes"].sum())
'''

In [ ]:
'''
df_total_spikes.reset_index(inplace=True)
'''

In [ ]:
# Start in position 1, not 0
'''
if mode == "4_pos":
    df_total_spikes.position = df_total_spikes.position+1
    df_total_spikes.position_attack = df_total_spikes.position_attack+1
'''

In [ ]:
# Dump the resulting dataframe to CSV

#df_total_spikes.to_csv(stats_dir+"total_spikes.csv", mode = "a", index = False, header=True, sep=";")

In [ ]:
df_total_spikes = pd.read_csv(stats_dir+"total_spikes.csv", delimiter=";")

In [ ]:
df_total_spikes

#### Horizontal dispersion

In [ ]:
#df_horizontal = df_aggr.copy()
'''
if mode == "all_pos":
    df_horizontal = df_aggr[df_aggr.n_neurons.isin([0,5,35,55,75,105])].copy()
elif mode == "4_pos":
    df_horizontal = df_aggr.copy()
else:
    df_horizontal = df_aggr.copy()
'''

In [ ]:
'''
if mode == "all_pos":
    df_horizontal = pd.DataFrame(df_horizontal.groupby(["attack_n_neurons", "position_attack", "position", "n_exec"])["time_delta"].count()/10000*100)
elif mode == "4_pos":
    df_horizontal = pd.DataFrame(df_horizontal.groupby(["attack_n_neurons", "position_attack", "position", "v_increment", "n_exec"])["time_delta"].count()/10000*100)
else:
    df_horizontal = pd.DataFrame(df_horizontal.groupby(["attack_n_neurons", "position_attack", "position", "n_exec"])["time_delta"].count()/10000*100)
'''    

In [ ]:
'''
df_horizontal.reset_index(inplace=True)
'''

In [ ]:
# Start in position 1, not 0
'''
if mode == "4_pos":
    df_horizontal.position = df_horizontal.position+1
    df_horizontal.position_attack = df_horizontal.position_attack+1
'''

In [ ]:
# Dump the resulting dataframe to CSV

#df_horizontal.to_csv(stats_dir+"horizontal_dispersion.csv", mode = "a", index = False, header=True, sep=";")

In [ ]:
df_horizontal = pd.read_csv(stats_dir+"horizontal_dispersion.csv", delimiter=";")

In [ ]:
df_horizontal

In [ ]:
# Merge both Brian dataframes and increment position values
df_Brian = df_total_spikes.copy()
df_Brian["dispersion"] = df_horizontal["time_delta"]

if mode == "all_pos":
    df_Brian["position"] = df_Brian["position"]+1
    df_Brian["position_attack"] = df_Brian["position_attack"]+1
    
if mode == "JAM_always_attack":
    df_Brian = df_Brian.drop(columns=['position_attack'])

In [ ]:
df_Brian

### Load and process CNN data

In [ ]:
if mode == "all_pos":
    dir_data = "/data/CNN/output/FLO_all_positions"
    csv_filenames = ["original.csv", "Flooding.csv"]
elif mode == "4_pos":
    dir_data = "/data/CNN/output/FLO_4_positions"
    csv_filenames = ["original.csv", "Flooding.csv"]
elif mode == "JAM_always_attack":
    dir_data = "/data/CNN/output/JAM_always_attack/10exec_1-20n"
    csv_filenames = ["original.csv", "Jamming.csv"]

In [ ]:
dir_data

In [ ]:
df = pd.DataFrame()

for filename in csv_filenames:
    filepath = os.path.join(dir_data, filename)
    
    if filename in ["original.csv"] and attack != "JAM":
        df = pd.concat([df, pd.read_csv(filepath, delimiter=";", usecols=["attack", "n_nodes", "X", "Y", "position","solution", "stim_value", "n_steps", "n_exec"])], sort=True)
    
    if filename in ["original.csv"] and attack == "JAM":
        df = pd.concat([df, pd.read_csv(filepath, delimiter=";", usecols=["attack", "n_nodes", "positions_attacked","solution", "stim_value", "n_steps", "n_exec"])], sort=True)

    if filename in ["Flooding.csv"]:
        df = pd.concat([df, pd.read_csv(filepath, delimiter=";", usecols=["attack", "n_nodes", "X", "Y", "position","solution", "stim_value", "n_steps", "n_exec"])], sort=True)
    
    if filename in ["Jamming.csv"]:
        df = pd.concat([df, pd.read_csv(filepath, delimiter=";", usecols=["attack", "n_nodes", "positions_attacked","solution", "stim_value", "n_steps", "n_exec"])], sort=True)
    
# Fill NaN cells with zero value
#df = df.fillna(0)

# Create column "coordinate" appending X and Y values
if attack != "JAM":
    df["coordinate"] = df["X"].map(str) + ","+ df["Y"].map(str)

# quickfix for old csvs where the bug with the number of steps was not corrected
if mode == "all_pos":
    df["n_steps"] = df["n_steps"]+df["position"]-1

df.solution = df.solution.replace(False, 0)
df.solution = df.solution.replace(True, 1)

In [ ]:
if mode == "4_pos":
    # fix results generated with increments of 5, instead of 10 (Brian)
    selected_nodes = [0] # keep originalCNN rows

    for nodes in range(5,110,10):
        selected_nodes.append(nodes)

    df = df[df.n_nodes.isin(selected_nodes)].copy()

In [ ]:
if mode == "all_pos":
    df_FLO_all_pos = df[(df.n_nodes.isin([0,5,35,55,75,105])) & (df.stim_value.isin([0, 0.6]))].copy()
elif mode == "4_pos":
    df_FLO_4_pos = df.copy()
else:
    #df_JAM_always = df[(df.n_nodes.isin([0,5,35,55,75,105]))].copy()
    df_JAM_always = df.copy()

#### Correlation between solution and n_steps

In [ ]:
if mode == "all_pos":
    correlation_solution_n_steps = df_FLO_all_pos.corr()["solution"]["n_steps"].round(4)
    print("Correlation between solution and n_steps: ", df_FLO_all_pos.corr()["solution"]["n_steps"].round(4))
elif mode == "4_pos":
    correlation_solution_n_steps = df_FLO_4_pos.corr()["solution"]["n_steps"].round(4)
    print("Correlation between solution and n_steps: ", df_FLO_4_pos.corr()["solution"]["n_steps"].round(4))
else:
    correlation_solution_n_steps = df_JAM_always.corr()["solution"]["n_steps"].round(4)
    print("Correlation between solution and n_steps: ", df_JAM_always.corr()["solution"]["n_steps"].round(4))

### Process the Brian and CNN dataframes and unify them

#### Process CNN data

In [ ]:
if mode == "all_pos":
    df_FLO_all_pos = df_FLO_all_pos.replace("original_CNN", "initial_state")
    df_FLO_all_pos = df_FLO_all_pos.drop(columns=['X', 'Y', 'solution', 'stim_value', "coordinate"])
    df_FLO_all_pos["attack_n_neurons"] = df_FLO_all_pos["attack"].astype("str") + " " + df_FLO_all_pos["n_nodes"].astype("str")
    df_FLO_all_pos = df_FLO_all_pos.drop(columns=['n_nodes', 'attack'])
    df_FLO_all_pos = df_FLO_all_pos.rename(columns={"position": "position_attack"})
    df_FLO_all_pos = df_FLO_all_pos.groupby(["attack_n_neurons", "position_attack"]).agg({'n_steps':'mean'}).round(4)
    df_FLO_all_pos.reset_index(inplace=True, drop=True)
    
elif mode == "4_pos":
    df_FLO_4_pos = df_FLO_4_pos.replace("original_CNN", "initial_state")
    df_FLO_4_pos = df_FLO_4_pos.drop(columns=['X', 'Y', 'solution', "coordinate"])
    df_FLO_4_pos["attack_n_neurons"] = df_FLO_4_pos["attack"].astype("str") + " " + df_FLO_4_pos["n_nodes"].astype("str")
    df_FLO_4_pos = df_FLO_4_pos.drop(columns=['n_nodes', 'attack'])
    df_FLO_4_pos = df_FLO_4_pos.rename(columns={"position": "position_attack"})
    df_FLO_4_pos = df_FLO_4_pos.groupby(["attack_n_neurons", "position_attack", "stim_value"]).agg({'n_steps':'mean'}).round(4)
    df_FLO_4_pos = df_FLO_4_pos.sort_values(by=['attack_n_neurons', 'position_attack', 'stim_value'])
    df_FLO_4_pos.reset_index(inplace=True, drop=True)

else:
    df_JAM_always = df_JAM_always.replace("original_CNN", "initial_state")
    df_JAM_always = df_JAM_always.drop(columns=['solution', 'stim_value'])
    df_JAM_always["attack_n_neurons"] = df_JAM_always["attack"].astype("str") + " " + df_JAM_always["n_nodes"].astype("str")
    df_JAM_always = df_JAM_always.drop(columns=['n_nodes', 'attack', 'positions_attacked'])
    df_JAM_always = df_JAM_always.groupby(["attack_n_neurons"]).agg({'n_steps':'mean'}).round(4)
    df_JAM_always.reset_index(inplace=True)

#### Merged df between Brian and CNN data

In [ ]:
if mode == "all_pos":
    df_merge = df_Brian.copy()
    df_merge = df_merge.groupby(["attack_n_neurons", "position_attack"]).agg({'number_spikes':'mean', 'dispersion':'mean'}).round(4)
    df_merge.reset_index(inplace=True)
    df_merge.sort_values(by=['attack_n_neurons','position_attack'])
    df_merge["n_steps"] = df_FLO_all_pos["n_steps"]
    df_merge["attack"] = df_merge["attack_n_neurons"].str.split(" ", n = 1, expand = True)[0]
    df_merge["n_neurons"] = df_merge["attack_n_neurons"].str.split(" ", n = 1, expand = True)[1].astype('int64')
    df_merge = df_merge.drop(columns=['attack_n_neurons'])
elif mode == "4_pos":
    df_merge = df_Brian.copy()
    df_merge = df_merge.groupby(["attack_n_neurons", "position_attack","v_increment"]).agg({'number_spikes':'mean', 'dispersion':'mean'}).round(4)
    df_merge.reset_index(inplace=True)
    df_merge.sort_values(by=['attack_n_neurons','position_attack','v_increment'])
    df_merge["n_steps"] = df_FLO_4_pos["n_steps"]
    df_merge["attack"] = df_merge["attack_n_neurons"].str.split(" ", n = 1, expand = True)[0]
    df_merge["n_neurons"] = df_merge["attack_n_neurons"].str.split(" ", n = 1, expand = True)[1].astype('int64')
    df_merge = df_merge.drop(columns=['attack_n_neurons'])
else:
    df_merge = df_Brian.copy()
    df_merge = df_merge.groupby(["attack_n_neurons"]).agg({'number_spikes':'mean', 'dispersion':'mean'}).round(4)
    df_merge.reset_index(inplace=True)
    df_merge["n_steps"] = df_JAM_always["n_steps"]
    df_merge["attack"] = df_merge["attack_n_neurons"].str.split(" ", n = 1, expand = True)[0]
    df_merge["n_neurons"] = df_merge["attack_n_neurons"].str.split(" ", n = 1, expand = True)[1].astype('int64')
    df_merge = df_merge.drop(columns=['attack_n_neurons'])

In [ ]:
df_merge

In [ ]:
df_merge.sort_values(by=['n_neurons'])

In [ ]:
df_merge.corr().round(2)

In [ ]:
df_merge.corr(method="spearman").round(2)

In [ ]:
sns.pairplot(df_merge)

#### Correlation number_spikes and dispersion

In [ ]:
print("Correlation between spikes and dispersion: ", df_merge.corr()["number_spikes"]["dispersion"].round(4))

## Correlation analysis

### Correlation with fixed n_neurons

In [ ]:
list_n_neurons = df_merge.n_neurons.unique().tolist()[:-1]

In [ ]:
df_corr_fixed_neurons = pd.DataFrame(columns=["attack", "n_neurons", "corr_steps_spikes", "corr_steps_dispersion", "corr_spikes_dispersion"])

for n_neurons in list_n_neurons:
    temp_corr = df_merge[(df_merge.n_neurons == n_neurons)].corr()
    
    attack = df_merge[(df_merge.n_neurons == n_neurons)].attack.unique()[0]
    corr_steps_spikes = temp_corr["n_steps"]["number_spikes"]
    corr_steps_dispersion = temp_corr["n_steps"]["dispersion"]
    corr_spikes_dispersion = temp_corr["number_spikes"]["dispersion"]

    df_corr_fixed_neurons = df_corr_fixed_neurons.append({
                            "attack": attack,
                            "n_neurons": n_neurons,  
                            "corr_steps_spikes": corr_steps_spikes, 
                            "corr_steps_dispersion": corr_steps_dispersion, 
                            "corr_spikes_dispersion": corr_spikes_dispersion
                           }, ignore_index=True)
    
    
df_corr_fixed_neurons = df_corr_fixed_neurons.round(4)

In [ ]:
df_merge

In [ ]:
df_Brian

In [ ]:
df_corr_fixed_neurons.sort_values(by=['n_neurons'])

#### Correlation n_steps and number_spikes

In [ ]:
plots_prefix = "correlation/"+mode+"/"

if not os.path.exists(plots_prefix):
    os.makedirs(plots_prefix)

In [ ]:
###################### Correlation n_steps and number_spikes ######################
attack = "FLO"

label_size = 45
ticks_size = 35
legend_size = 35
line_size = 15

fig, ax = plt.subplots(figsize=(40,20))

if attack in ["FLO", "JAM"]:
    sns.lineplot(x="n_neurons", y="corr_steps_spikes", data=df_corr_fixed_neurons, linewidth=line_size, ax=ax) # ci=None 

    ax.set_xlabel("Number of neurons", fontsize=label_size, fontweight="bold")
    ax.set_ylabel("Correlation steps-spikes", fontsize=label_size, fontweight="bold")
    ax.tick_params(labelsize=ticks_size)

    ax.grid(b=True, which='major', color='#d8dcd6', linewidth=1.0)
    ax.grid(b=True, which='minor', color='#d8dcd6', linewidth=0.5)
    
    ax.set_xticks(list_n_neurons)

    leg = ax.legend(loc='best', fancybox=True, ncol=1, frameon='True',fontsize=legend_size)

fig.savefig(plots_prefix+"fixed_neurons-corr_steps_spikes.pdf")
plt.close()

#### Correlation n_steps and dispersion

In [ ]:
###################### Correlation n_steps and number_spikes ######################
attack = "FLO"

label_size = 45
ticks_size = 35
legend_size = 35
line_size = 15

fig, ax = plt.subplots(figsize=(40,20))

if attack in ["FLO", "JAM"]:
    sns.lineplot(x="n_neurons", y="corr_steps_dispersion", data=df_corr_fixed_neurons, linewidth=line_size, ax=ax) # ci=None 

    ax.set_xlabel("Number of neurons", fontsize=label_size, fontweight="bold")
    ax.set_ylabel("Correlation steps-dispersion", fontsize=label_size, fontweight="bold")
    ax.tick_params(labelsize=ticks_size)

    ax.grid(b=True, which='major', color='#d8dcd6', linewidth=1.0)
    ax.grid(b=True, which='minor', color='#d8dcd6', linewidth=0.5)
    
    ax.set_xticks(list_n_neurons)

    leg = ax.legend(loc='best', fancybox=True, ncol=1, frameon='True',fontsize=legend_size)

fig.savefig(plots_prefix+"fixed_neurons-corr_steps_dispersion.pdf")
plt.close()

#### Correlation number_spikes and dispersion

In [ ]:
###################### Correlation n_steps and number_spikes ######################
attack = "FLO"

label_size = 45
ticks_size = 35
legend_size = 35
line_size = 15

fig, ax = plt.subplots(figsize=(40,20))

if attack in ["FLO", "JAM"]:
    sns.lineplot(x="n_neurons", y="corr_spikes_dispersion", data=df_corr_fixed_neurons, linewidth=line_size, ax=ax) # ci=None 

    ax.set_xlabel("Number of neurons", fontsize=label_size, fontweight="bold")
    ax.set_ylabel("Correlation spikes-dispersion", fontsize=label_size, fontweight="bold")
    ax.tick_params(labelsize=ticks_size)

    ax.grid(b=True, which='major', color='#d8dcd6', linewidth=1.0)
    ax.grid(b=True, which='minor', color='#d8dcd6', linewidth=0.5)
    
    ax.set_xticks(list_n_neurons)

    leg = ax.legend(loc='best', fancybox=True, ncol=1, frameon='True',fontsize=legend_size)

fig.savefig(plots_prefix+"fixed_neurons-corr_spikes_dispersion.pdf")
plt.close()

### Correlation with fixed positions

In [ ]:
plots_prefix = "correlation/"+mode+"/"

if not os.path.exists(plots_prefix):
    os.makedirs(plots_prefix)

In [ ]:
list_positions = df_merge.position_attack.unique().tolist()

In [ ]:
df_corr_fixed_positions = pd.DataFrame(columns=["attack", "position_attack", "corr_steps_spikes", "corr_steps_dispersion", "corr_spikes_dispersion"])

for position in list_positions:
    temp_corr = df_merge[(df_merge.position_attack == position)].corr()
    
    attack = df_merge[(df_merge.position_attack == position)].attack.unique()[0]
    corr_steps_spikes = temp_corr["n_steps"]["number_spikes"]
    corr_steps_dispersion = temp_corr["n_steps"]["dispersion"]
    corr_spikes_dispersion = temp_corr["number_spikes"]["dispersion"]

    df_corr_fixed_positions = df_corr_fixed_positions.append({
                            "attack": attack,
                            "position_attack": position,  
                            "corr_steps_spikes": corr_steps_spikes, 
                            "corr_steps_dispersion": corr_steps_dispersion, 
                            "corr_spikes_dispersion": corr_spikes_dispersion
                           }, ignore_index=True)
    
    
df_corr_fixed_positions = df_corr_fixed_positions.round(4)

In [ ]:
df_corr_fixed_positions

#### Correlation n_steps and number_spikes

In [ ]:
###################### Correlation n_steps and number_spikes ######################
attack = "FLO"

label_size = 45
ticks_size = 35
legend_size = 35
line_size = 15

fig, ax = plt.subplots(figsize=(40,20))

if attack in ["FLO", "JAM"]:
    sns.lineplot(x="position_attack", y="corr_steps_spikes", data=df_corr_fixed_positions, linewidth=line_size, ax=ax) # ci=None 

    ax.set_xlabel("Position of attack", fontsize=label_size, fontweight="bold")
    ax.set_ylabel("Correlation steps-spikes", fontsize=label_size, fontweight="bold")
    ax.tick_params(labelsize=ticks_size)

    ax.grid(b=True, which='major', color='#d8dcd6', linewidth=1.0)
    ax.grid(b=True, which='minor', color='#d8dcd6', linewidth=0.5)
    
    ax.set_xticks(list_positions)

    leg = ax.legend(loc='best', fancybox=True, ncol=1, frameon='True',fontsize=legend_size)

fig.savefig(plots_prefix+"fixed_positions-corr_steps_spikes.pdf")
plt.close()

#### Correlation n_steps and dispersion

In [ ]:
###################### Correlation n_steps and number_spikes ######################
attack = "FLO"

label_size = 45
ticks_size = 35
legend_size = 35
line_size = 15

fig, ax = plt.subplots(figsize=(40,20))

if attack in ["FLO", "JAM"]:
    sns.lineplot(x="position_attack", y="corr_steps_dispersion", data=df_corr_fixed_positions, linewidth=line_size, ax=ax) # ci=None 

    ax.set_xlabel("Position of attack", fontsize=label_size, fontweight="bold")
    ax.set_ylabel("Correlation steps-dispersion", fontsize=label_size, fontweight="bold")
    ax.tick_params(labelsize=ticks_size)

    ax.grid(b=True, which='major', color='#d8dcd6', linewidth=1.0)
    ax.grid(b=True, which='minor', color='#d8dcd6', linewidth=0.5)
    
    ax.set_xticks(list_positions)

    leg = ax.legend(loc='best', fancybox=True, ncol=1, frameon='True',fontsize=legend_size)

fig.savefig(plots_prefix+"fixed_positions-corr_steps_dispersion.pdf")
plt.close()

#### Correlation number_spikes and dispersion

In [ ]:
###################### Correlation n_steps and number_spikes ######################
attack = "FLO"

label_size = 45
ticks_size = 35
legend_size = 35
line_size = 15

fig, ax = plt.subplots(figsize=(40,20))

if attack in ["FLO", "JAM"]:
    sns.lineplot(x="position_attack", y="corr_spikes_dispersion", data=df_corr_fixed_positions, linewidth=line_size, ax=ax) # ci=None 

    ax.set_xlabel("Position of attack", fontsize=label_size, fontweight="bold")
    ax.set_ylabel("Correlation spikes-dispersion", fontsize=label_size, fontweight="bold")
    ax.tick_params(labelsize=ticks_size)

    ax.grid(b=True, which='major', color='#d8dcd6', linewidth=1.0)
    ax.grid(b=True, which='minor', color='#d8dcd6', linewidth=0.5)
    
    ax.set_xticks(list_positions)

    leg = ax.legend(loc='best', fancybox=True, ncol=1, frameon='True',fontsize=legend_size)

fig.savefig(plots_prefix+"fixed_positions-corr_spikes_dispersion.pdf")
plt.close()

### Correlation with fixed v_increment (ONLY FOR ZOOM IN 4 POS)

In [ ]:
list_v_increments = df_merge.v_increment.unique().tolist()[:-1]

In [ ]:
df_corr_fixed_increments = pd.DataFrame(columns=["attack", "v_increment", "corr_steps_spikes", "corr_steps_dispersion", "corr_spikes_dispersion"])

for v_increment in list_v_increments:
    temp_corr = df_merge[(df_merge.v_increment == v_increment)].corr()
    
    attack = df_merge[(df_merge.v_increment == v_increment)].attack.unique()[0]
    corr_steps_spikes = temp_corr["n_steps"]["number_spikes"]
    corr_steps_dispersion = temp_corr["n_steps"]["dispersion"]
    corr_spikes_dispersion = temp_corr["number_spikes"]["dispersion"]

    df_corr_fixed_increments = df_corr_fixed_increments.append({
                            "attack": attack,
                            "v_increment": v_increment,  
                            "corr_steps_spikes": corr_steps_spikes, 
                            "corr_steps_dispersion": corr_steps_dispersion, 
                            "corr_spikes_dispersion": corr_spikes_dispersion
                           }, ignore_index=True)
    
    
df_corr_fixed_increments = df_corr_fixed_increments.round(4)

In [ ]:
df_corr_fixed_increments.sort_values(by=['v_increment'])

#### Correlation n_steps and number_spikes

In [ ]:
plots_prefix = "correlation/"+mode+"/"

if not os.path.exists(plots_prefix):
    os.makedirs(plots_prefix)

In [ ]:
###################### Correlation n_steps and number_spikes ######################
attack = "FLO"

label_size = 45
ticks_size = 35
legend_size = 35
line_size = 15

fig, ax = plt.subplots(figsize=(40,20))

if attack in ["FLO", "JAM"]:
    sns.lineplot(x="v_increment", y="corr_steps_spikes", data=df_corr_fixed_increments, linewidth=line_size, ax=ax) # ci=None 

    ax.set_xlabel("Voltage increment", fontsize=label_size, fontweight="bold")
    ax.set_ylabel("Correlation steps-spikes", fontsize=label_size, fontweight="bold")
    ax.tick_params(labelsize=ticks_size)

    ax.grid(b=True, which='major', color='#d8dcd6', linewidth=1.0)
    ax.grid(b=True, which='minor', color='#d8dcd6', linewidth=0.5)
    
    ax.set_xticks(list_v_increments)

    leg = ax.legend(loc='best', fancybox=True, ncol=1, frameon='True',fontsize=legend_size)

fig.savefig(plots_prefix+"fixed_v_increment-corr_steps_spikes.pdf")
plt.close()

#### Correlation n_steps and dispersion

In [ ]:
###################### Correlation n_steps and number_spikes ######################
attack = "FLO"

label_size = 45
ticks_size = 35
legend_size = 35
line_size = 15

fig, ax = plt.subplots(figsize=(40,20))

if attack in ["FLO", "JAM"]:
    sns.lineplot(x="v_increment", y="corr_steps_dispersion", data=df_corr_fixed_increments, linewidth=line_size, ax=ax) # ci=None 

    ax.set_xlabel("Voltage increment", fontsize=label_size, fontweight="bold")
    ax.set_ylabel("Correlation steps-dispersion", fontsize=label_size, fontweight="bold")
    ax.tick_params(labelsize=ticks_size)

    ax.grid(b=True, which='major', color='#d8dcd6', linewidth=1.0)
    ax.grid(b=True, which='minor', color='#d8dcd6', linewidth=0.5)
    
    ax.set_xticks(list_v_increments)

    leg = ax.legend(loc='best', fancybox=True, ncol=1, frameon='True',fontsize=legend_size)

fig.savefig(plots_prefix+"fixed_v_increment-corr_steps_dispersion.pdf")
plt.close()

#### Correlation number_spikes and dispersion

In [ ]:
###################### Correlation n_steps and number_spikes ######################
attack = "FLO"

label_size = 45
ticks_size = 35
legend_size = 35
line_size = 15

fig, ax = plt.subplots(figsize=(40,20))

if attack in ["FLO", "JAM"]:
    sns.lineplot(x="v_increment", y="corr_spikes_dispersion", data=df_corr_fixed_increments, linewidth=line_size, ax=ax) # ci=None 

    ax.set_xlabel("Voltage increment", fontsize=label_size, fontweight="bold")
    ax.set_ylabel("Correlation spikes-dispersion", fontsize=label_size, fontweight="bold")
    ax.tick_params(labelsize=ticks_size)

    ax.grid(b=True, which='major', color='#d8dcd6', linewidth=1.0)
    ax.grid(b=True, which='minor', color='#d8dcd6', linewidth=0.5)
    
    ax.set_xticks(list_v_increments)

    leg = ax.legend(loc='best', fancybox=True, ncol=1, frameon='True',fontsize=legend_size)

fig.savefig(plots_prefix+"fixed_v_increment-corr_spikes_dispersion.pdf")
plt.close()

In [ ]:
df_corr_results = pd.DataFrame(columns=["attack", "n_neurons", "position_attack", "corr_steps_spikes", "corr_steps_dispersion", "corr_spikes_dispersion"])

In [ ]:
list_n_neurons = df_merge.attack_n_neurons.unique().tolist()[:-1]
list_positions = df_merge.position_attack.unique().tolist()

In [ ]:

for attack_n_neurons in list_attack_n_neurons:
    for position_attack in list_positions:
        temp_corr = df_merge[(df_merge.n_neurons == n_neurons) & (df_merge.position_attack == position_attack)].corr()
        
        attack = attack_n_neurons.split(" ")[0]
        n_neurons = attack_n_neurons.split(" ")[1]
        corr_steps_spikes = temp_corr["n_steps"]["number_spikes"]
        corr_steps_dispersion = temp_corr["n_steps"]["dispersion"]
        corr_spikes_dispersion = temp_corr["number_spikes"]["dispersion"]
        
        df_corr_results = df_corr_results.append({"attack": attack,
                                "n_neurons": n_neurons, 
                                "position_attack": position_attack, 
                                "corr_steps_spikes": corr_steps_spikes, 
                                "corr_steps_dispersion": corr_steps_dispersion, 
                                "corr_spikes_dispersion": corr_spikes_dispersion
                               }, ignore_index=True)
                               

In [ ]:
df_corr_results[df_corr_results.position_attack == 1]

In [ ]:
list_positions = df_corr_results.position_attack.unique().tolist()

if mode == "all_pos":
    list_n_neurons = ['5', '35', '55', '75', '105']

In [ ]:
###################### Correlation n_steps and number_spikes ######################
attack = "FLO"

label_size = 45
ticks_size = 35
legend_size = 35
line_size = 15

fig, ax = plt.subplots(figsize=(40,20))

if attack in ["FLO", "JAM"]:
    for n_neurons in list_n_neurons:
        sns.lineplot(x="position_attack", y="corr_steps_spikes", data=df_corr_results[df_corr_results.n_neurons == n_neurons], linewidth=line_size, ax=ax, label=str(n_neurons)+"n") # ci=None 

    ax.set_xlabel("Position of attack", fontsize=label_size, fontweight="bold")
    ax.set_ylabel("Correlation steps-spikes", fontsize=label_size, fontweight="bold")
    ax.tick_params(labelsize=ticks_size)

    ax.grid(b=True, which='major', color='#d8dcd6', linewidth=1.0)
    ax.grid(b=True, which='minor', color='#d8dcd6', linewidth=0.5)
    
    ax.set_xticks(list_positions)

    leg = ax.legend(loc='best', fancybox=True, ncol=1, frameon='True',fontsize=legend_size)

#fig.savefig(plots_prefix_dispersion+"total_number_spikes.pdf")
#plt.close()

In [ ]:
###################### Correlation n_steps and dispersion ######################
attack = "FLO"

label_size = 45
ticks_size = 35
legend_size = 35
line_size = 15

fig, ax = plt.subplots(figsize=(40,20))

if attack in ["FLO", "JAM"]:
    for n_neurons in list_n_neurons:
        sns.lineplot(x="position_attack", y="corr_steps_dispersion", data=df_corr_results[df_corr_results.n_neurons == n_neurons], linewidth=line_size, ax=ax, label=str(n_neurons)+"n") # ci=None 

    ax.set_xlabel("Position of attack", fontsize=label_size, fontweight="bold")
    ax.set_ylabel("Correlation steps-dispersion", fontsize=label_size, fontweight="bold")
    ax.tick_params(labelsize=ticks_size)

    ax.grid(b=True, which='major', color='#d8dcd6', linewidth=1.0)
    ax.grid(b=True, which='minor', color='#d8dcd6', linewidth=0.5)
    
    ax.set_xticks(list_positions)

    leg = ax.legend(loc='best', fancybox=True, ncol=1, frameon='True',fontsize=legend_size)

#fig.savefig(plots_prefix_dispersion+"total_number_spikes.pdf")
#plt.close()

In [ ]:
###################### Correlation number_spikes and dispersion ######################
attack = "FLO"

label_size = 45
ticks_size = 35
legend_size = 35
line_size = 15

fig, ax = plt.subplots(figsize=(40,20))

if attack in ["FLO", "JAM"]:
    for n_neurons in list_n_neurons:
        sns.lineplot(x="position_attack", y="corr_spikes_dispersion", data=df_corr_results[df_corr_results.n_neurons == n_neurons], linewidth=line_size, ax=ax, label=str(n_neurons)+"n") # ci=None 

    ax.set_xlabel("Position of attack", fontsize=label_size, fontweight="bold")
    ax.set_ylabel("Correlation spikes-dispersion", fontsize=label_size, fontweight="bold")
    ax.tick_params(labelsize=ticks_size)

    ax.grid(b=True, which='major', color='#d8dcd6', linewidth=1.0)
    ax.grid(b=True, which='minor', color='#d8dcd6', linewidth=0.5)
    
    ax.set_xticks(list_positions)

    leg = ax.legend(loc='best', fancybox=True, ncol=1, frameon='True',fontsize=legend_size)

#fig.savefig(plots_prefix_dispersion+"total_number_spikes.pdf")
#plt.close()